
# 🔍 Khám phá dữ liệu MIMIC‑IV (đọc nhẹ, không giải nén toàn bộ)
Notebook này giúp bạn **xem nhanh** và **duyệt theo từng phần** các file `.csv.gz` cực lớn của MIMIC‑IV / MIMIC‑IV‑Note **mà không cần giải nén**, để tránh đầy bộ nhớ.

**Bạn có thể:**
- Xem vài dòng đầu (`nrows`) của từng bảng.
- Đọc theo **chunk** nhỏ (từng lô 5k–50k dòng) để kiểm tra dữ liệu.
- Lấy **mẫu ngẫu nhiên** một số dòng từ file cực lớn (bằng *reservoir sampling* trên các chunk).

> 💡 Phù hợp cho MacBook Air M2 / máy RAM thấp.



## 0) Cấu hình đường dẫn dữ liệu
Điền đúng thư mục gốc chứa 2 cây thư mục sau (theo bạn cung cấp):

```
data/
├── mimic-iv-note/2.2/note/*.csv(.gz)
└── mimiciv/3.1/{hosp,icu}/*.csv.gz
```


In [1]:
from pathlib import Path

# Lùi một cấp ra khỏi notebooks/, rồi vào data/
DATA_ROOT = Path("..") / "data"

HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
ICU_DIR  = DATA_ROOT / "mimiciv" / "3.1" / "icu"
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"

for p in [HOSP_DIR, ICU_DIR, NOTE_DIR]:
    print(f"Tồn tại: {p.resolve()} ->", p.exists())


Tồn tại: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv/3.1/hosp -> True
Tồn tại: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv/3.1/icu -> True
Tồn tại: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimic-iv-note/2.2/note -> True



## 1) Liệt kê nhanh các file `.csv.gz`


In [2]:

import glob

def list_gz(dir_path):
    files = sorted(glob.glob(str(dir_path / "*.csv.gz")))
    for f in files:
        print("•", Path(f).name)
    return files

print("📂 HOSP:")
_ = list_gz(HOSP_DIR)
print("\n📂 ICU:")
_ = list_gz(ICU_DIR)
print("\n📂 NOTE:")
_ = list_gz(NOTE_DIR)


📂 HOSP:
• admissions.csv.gz
• d_hcpcs.csv.gz
• d_icd_diagnoses.csv.gz
• d_icd_procedures.csv.gz
• d_labitems.csv.gz
• diagnoses_icd.csv.gz
• drgcodes.csv.gz
• emar.csv.gz
• emar_detail.csv.gz
• hcpcsevents.csv.gz
• labevents.csv.gz
• microbiologyevents.csv.gz
• omr.csv.gz
• patients.csv.gz
• pharmacy.csv.gz
• poe.csv.gz
• poe_detail.csv.gz
• prescriptions.csv.gz
• procedures_icd.csv.gz
• provider.csv.gz
• services.csv.gz
• transfers.csv.gz

📂 ICU:
• caregiver.csv.gz
• chartevents.csv.gz
• d_items.csv.gz
• datetimeevents.csv.gz
• icustays.csv.gz
• ingredientevents.csv.gz
• inputevents.csv.gz
• outputevents.csv.gz
• procedureevents.csv.gz

📂 NOTE:
• discharge.csv.gz
• discharge_detail.csv.gz
• radiology.csv.gz
• radiology_detail.csv.gz



## 2) Hàm tiện ích đọc **nhẹ** (không giải nén toàn bộ)
- `preview_csv(path, nrows)`: xem **n dòng đầu** trực tiếp từ `.csv.gz`  
- `read_in_chunks(path, chunksize)`: duyệt **từng chunk** để kiểm tra cấu trúc/giá trị  
- `show_columns(path)`: chỉ đọc **1 dòng** để lấy **tên cột**
- `sample_rows(path, k, chunksize)`: **lấy mẫu ngẫu nhiên** `k` dòng từ file rất lớn (reservoir sampling)


In [ ]:

import pandas as pd
import random

def preview_csv(path, nrows=5):
    df = pd.read_csv(path, compression="gzip", nrows=nrows, low_memory=False)
    print(f"📄 {path} — hiển thị {nrows} dòng đầu:")
    display(df)
    return df

def read_in_chunks(path, chunksize=10000, max_chunks=2, head_each=3):
    i = 0
    for chunk in pd.read_csv(path, compression="gzip", chunksize=chunksize, low_memory=False):
        print(f"🔹 Chunk {i} (khoảng {len(chunk)} dòng) — xem {head_each} dòng đầu:")
        display(chunk.head(head_each))
        i += 1
        if i >= max_chunks:
            break

def show_columns(path):
    df = pd.read_csv(path, compression="gzip", nrows=1, low_memory=False)
    print(f"📑 Cột trong {path}:")
    print(list(df.columns))
    return df.columns.tolist()

def sample_rows(path, k=1000, chunksize=50000, random_state=42):
    """Reservoir sampling cho file cực lớn: chọn ngẫu nhiên k dòng mà không cần load toàn bộ."""
    random.seed(random_state)
    reservoir = None
    total = 0
    for chunk in pd.read_csv(path, compression="gzip", chunksize=chunksize, low_memory=False):
        # lấy ngẫu nhiên các index trong chunk để thêm vào reservoir
        if reservoir is None:
            reservoir = chunk.sample(min(k, len(chunk)), random_state=random_state)
        else:
            m = len(chunk)
            take = min(k, m)
            # với mỗi hàng của chunk, xác suất giữ lại ~ k/(total + i)
            # để đơn giản & hiệu quả, dùng sampling theo tỷ lệ
            add = chunk.sample(take, random_state=random_state)
            combined = pd.concat([reservoir, add], ignore_index=True)
            reservoir = combined.sample(min(k, len(combined)), random_state=random_state)
        total += len(chunk)
    print(f"✅ Lấy mẫu {len(reservoir)} dòng ngẫu nhiên từ tổng ~{total} dòng.")
    return reservoir



## 3) Ví dụ nhanh trên HOSP (admissions, patients, diagnoses_icd, labevents, d_labitems)
> Bạn có thể chạy từng cell dưới đây tuỳ nhu cầu.


In [ ]:

admissions_path = HOSP_DIR / "admissions.csv.gz"
patients_path   = HOSP_DIR / "patients.csv.gz"
diagnoses_path  = HOSP_DIR / "diagnoses_icd.csv.gz"
labevents_path  = HOSP_DIR / "labevents.csv.gz"
dlab_path       = HOSP_DIR / "d_labitems.csv.gz"

preview_csv(admissions_path, nrows=1000000)
show_columns(admissions_path)


In [ ]:

preview_csv(patients_path, nrows=5)
show_columns(patients_path)


In [ ]:

preview_csv(diagnoses_path, nrows=5)
show_columns(diagnoses_path)


In [ ]:

# ⚠️ labevents rất lớn — chỉ xem 5 dòng + duyệt 1-2 chunk nhỏ
preview_csv(labevents_path, nrows=5)
read_in_chunks(labevents_path, chunksize=50000, max_chunks=1, head_each=5)
show_columns(labevents_path)


In [ ]:

preview_csv(dlab_path, nrows=5)
show_columns(dlab_path)



## 4) Ví dụ nhanh trên NOTE (discharge, radiology)


In [ ]:
import pandas as pd

# Đọc một phần nhỏ để không nặng máy
df_dis = pd.read_csv(
    discharge_path,
    usecols=["hadm_id", "charttime", "text"],
    compression="gzip",
    nrows=3
)

# Hiển thị toàn bộ nội dung mà không bị rút gọn
pd.set_option("display.max_colwidth", None)

for i, row in df_dis.iterrows():
    print("=" * 80)
    print(f"HADM_ID: {row['hadm_id']} | CHARTTIME: {row['charttime']}")
    print("-" * 80)
    print(row["text"])
    print()


In [4]:
import pandas as pd
from pathlib import Path

HOSP_DIR = Path("../data/mimiciv/3.1/hosp")

proc = pd.read_csv(HOSP_DIR / "procedures_icd.csv.gz", compression="gzip")
dproc = pd.read_csv(HOSP_DIR / "d_icd_procedures.csv.gz", compression="gzip")

merged = proc.merge(dproc, on=["icd_code", "icd_version"], how="left")

imaging_kw = [
    "x-ray", "radiograph", "ct", "mri", "ultrasound",
    "fluoroscopy", "angiography", "echocardiography", "imaging"
]

pattern = "|".join(imaging_kw)
imaging_codes = merged[merged["long_title"].str.lower().str.contains(pattern, na=False)]

display(imaging_codes[["hadm_id", "icd_version", "icd_code", "long_title"]].head(20))


,hadm_id,icd_version,icd_code,long_title
6,28979390,9,5551,Nephroureterectomy
7,26134563,9,3734,Excision or destruction of other lesion or tis...
8,26134563,9,3728,Intracardiac echocardiography
22,29541074,9,4562,Other partial resection of small intestine
45,22119639,9,4576,Open and other sigmoidectomy
49,21544441,10,0TTB4ZZ,"Resection of Bladder, Percutaneous Endoscopic ..."
51,21544441,10,0UT9FZZ,"Resection of Uterus, Via Natural or Artificial..."
52,21544441,10,0UTC7ZZ,"Resection of Cervix, Via Natural or Artificial..."
53,21544441,10,0UT2FZZ,"Resection of Bilateral Ovaries, Via Natural or..."
58,26840593,10,3E0436Z,Introduction of Nutritional Substance into Cen...


In [5]:
import pandas as pd
from pathlib import Path

# ======== Cấu hình đường dẫn ========
DATA_ROOT = Path("../data")
HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"

# ======== 1️⃣ Tổng số lượt nhập viện ========
adm = pd.read_csv(HOSP_DIR / "admissions.csv.gz", usecols=["hadm_id"], compression="gzip")
total_hadm = adm["hadm_id"].nunique()

# ======== 2️⃣ Số lượt nhập viện có note xuất viện ========
discharge = pd.read_csv(
    NOTE_DIR / "discharge.csv.gz",
    usecols=["hadm_id"],
    compression="gzip"
)
hadm_with_discharge = discharge["hadm_id"].dropna().astype(int).nunique()

# ======== 3️⃣ Số lượt nhập viện có thủ thuật (procedures_icd) ========
procedures = pd.read_csv(
    HOSP_DIR / "procedures_icd.csv.gz",
    usecols=["hadm_id"],
    compression="gzip"
)
hadm_with_proc = procedures["hadm_id"].dropna().astype(int).nunique()

# ======== 4️⃣ Số lượt nhập viện có xét nghiệm (labevents) ========
labs = pd.read_csv(
    HOSP_DIR / "labevents.csv.gz",
    usecols=["hadm_id"],
    compression="gzip"
)
hadm_with_lab = labs["hadm_id"].dropna().astype(int).nunique()

# ======== 5️⃣ In thống kê ========
print("=== TỔNG QUAN DỮ LIỆU THEO LẦN NHẬP VIỆN ===")
print(f"- Tổng số lượt nhập viện (hadm_id): {total_hadm:,}")
print(f"- Có ghi chú xuất viện (discharge note): {hadm_with_discharge:,} "
      f"({hadm_with_discharge/total_hadm:.1%})")
print(f"- Có thủ thuật (procedures_icd): {hadm_with_proc:,} "
      f"({hadm_with_proc/total_hadm:.1%})")
print(f"- Có xét nghiệm (labevents): {hadm_with_lab:,} "
      f"({hadm_with_lab/total_hadm:.1%})")


=== TỔNG QUAN DỮ LIỆU THEO LẦN NHẬP VIỆN ===
- Tổng số lượt nhập viện (hadm_id): 546,028
- Có ghi chú xuất viện (discharge note): 331,793 (60.8%)
- Có thủ thuật (procedures_icd): 287,504 (52.7%)
- Có xét nghiệm (labevents): 447,689 (82.0%)


In [1]:
import pandas as pd
from pathlib import Path

# ====== Cấu hình đường dẫn ======
DATA_ROOT = Path("../data")
HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"

# ====== 1️⃣ Tổng số lượt nhập viện ======
adm = pd.read_csv(HOSP_DIR / "admissions.csv.gz", usecols=["hadm_id"], compression="gzip")
total_hadm = adm["hadm_id"].dropna().astype(int).nunique()

# ====== 2️⃣ Các nhóm chính ======
hadm_with_discharge = set(
    pd.read_csv(NOTE_DIR / "discharge.csv.gz", usecols=["hadm_id"], compression="gzip")
      ["hadm_id"].dropna().astype(int)
)
hadm_with_icd = set(
    pd.read_csv(HOSP_DIR / "diagnoses_icd.csv.gz", usecols=["hadm_id"], compression="gzip")
      ["hadm_id"].dropna().astype(int)
)
hadm_with_proc = set(
    pd.read_csv(HOSP_DIR / "procedures_icd.csv.gz", usecols=["hadm_id"], compression="gzip")
      ["hadm_id"].dropna().astype(int)
)
hadm_with_lab = set(
    pd.read_csv(HOSP_DIR / "labevents.csv.gz", usecols=["hadm_id"], compression="gzip")
      ["hadm_id"].dropna().astype(int)
)

# ====== 3️⃣ Giao giữa các nhóm ======
has_all = hadm_with_discharge & hadm_with_icd & hadm_with_proc & hadm_with_lab

# ====== 4️⃣ In thống kê ======
print("=== THỐNG KÊ DỮ LIỆU GỐC MIMIC-IV THEO LẦN NHẬP VIỆN ===")
print(f"- Tổng số lượt nhập viện: {total_hadm:,}")
print(f"- Có ghi chú xuất viện:   {len(hadm_with_discharge):,} ({len(hadm_with_discharge)/total_hadm:.1%})")
print(f"- Có chẩn đoán (ICD):     {len(hadm_with_icd):,} ({len(hadm_with_icd)/total_hadm:.1%})")
print(f"- Có thủ thuật (PROC):    {len(hadm_with_proc):,} ({len(hadm_with_proc)/total_hadm:.1%})")
print(f"- Có xét nghiệm (LAB):    {len(hadm_with_lab):,} ({len(hadm_with_lab)/total_hadm:.1%})")
print()
print(f"➡️  Có đủ cả 4 (Discharge + ICD + Procedure + Lab): {len(has_all):,} ({len(has_all)/total_hadm:.1%})")


=== THỐNG KÊ DỮ LIỆU GỐC MIMIC-IV THEO LẦN NHẬP VIỆN ===
- Tổng số lượt nhập viện: 546,028
- Có ghi chú xuất viện:   331,793 (60.8%)
- Có chẩn đoán (ICD):     545,497 (99.9%)
- Có thủ thuật (PROC):    287,504 (52.7%)
- Có xét nghiệm (LAB):    447,689 (82.0%)

➡️  Có đủ cả 4 (Discharge + ICD + Procedure + Lab): 186,014 (34.1%)



## 5) Lấy **mẫu ngẫu nhiên** từ file rất lớn (không load toàn bộ)
Ví dụ: lấy 2.000 dòng ngẫu nhiên từ `labevents.csv.gz`, sau đó lưu ra file nhỏ để xem nhanh.


In [ ]:

sample = sample_rows(labevents_path, k=2000, chunksize=100000, random_state=7)
out_path = DATA_ROOT / "sample_labevents_2000rows.csv"
sample.to_csv(out_path, index=False)
print("💾 Đã lưu:", out_path)
display(sample.head(10))



## 6) Mẹo tiết kiệm bộ nhớ
- Dùng `nrows` để xem nhanh vài dòng đầu, **không** giải nén toàn bộ.
- Dùng `chunksize` để **duyệt từng phần**, tránh đầy RAM.
- Chỉ chọn **cột cần thiết** bằng `usecols=[...]` nếu bạn biết sẵn danh sách cột.
- Lưu **mẫu nhỏ** ra `.csv` để thử nghiệm pipeline trước khi xử lý toàn bộ dữ liệu.
- Tránh `.read_csv(..., low_memory=True)` với file nhiều kiểu dữ liệu; dùng `low_memory=False` để pandas đoán kiểu ổn định hơn (ít lỗi).
